In [6]:
import sys
import os
from scipy import linalg
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import plotly.graph_objects as go
sys.path.append("..")

from utils.camera_utils import *

In [7]:
run_name = "RUN5"

spiegel_calib = ["spiegel1.mp4", "spiegel3.mp4", "spiegel4.mp4", "spiegel5.mp4", "spiegel6.mp4", "spiegel7.mp4", "spiegel8.mp4",
                "spiegel9.mp4", "spiegel10.mp4"]
front_calib = ["front1.mp4", "front2.mp4", "front3.mp4", "front4.mp4", "front5.mp4", "front6.mp4", "front7.mp4", "front8.mp4", 
              "front9.mp4", "front10.mp4"]

stereo_calib = ["stereo1.mp4", "stereo2.mp4", "stereo3.mp4", "stereo4.mp4", "stereo5.mp4"]

stereo_vali = ["stereo6.mp4", "stereo8.mp4", "stereo9.mp4"]

In [8]:
calibration_dir = f"../videos/{run_name}/calibration"
detection_dir = f"../videos/{run_name}/detection"
detection_files = os.listdir(detection_dir)
sC = stereoCamera(name=run_name, camera_size={0: (360, 180), 1: (360, 180)})
vL = videoLoader()


# Camera Positions

In [10]:
vL.load_video(os.path.join(detection_dir, detection_files[-1]))
frame = vL[10]
# 0 = mirror-cam 1 = frontal-cam
scaling_factor = 1 # image gets resized by this factor

sC.set_anchor_point(frame, 0, scaling_factor)
sC.set_anchor_point(frame, 1, scaling_factor)

# Displaying the camera regions for evaluation
for frame in vL[:100]:
    frame = sC.draw_camera_region(frame)
    cv2.imshow("Frame", frame)
    frame0, frame1 = sC(frame)
    cv2.waitKey(10)
cv2.destroyAllWindows()


In [12]:
sC.save_to_yaml(f"{run_name}cam_positions.yaml")

# Mirror Camera Calibration

In [5]:
frames = []
for file in spiegel_calib:
    vL.load_video(os.path.join(calibration_dir, file))
    video_section = vL[100:110]
    if len(video_section) > 0:
        frames.append(video_section)
    
sC.calibrate(frames,0, rows=8, columns=10, fallback_manual=True, image_scaling=2)

AssertionError: No Corners detected!

# Frontal Camera Calibration

In [ ]:


frames = []
for file in front_calib:
    vL.load_video(os.path.join(calibration_dir, file))
    frames.append(vL[100:110])
      
sC.calibrate(frames,1, rows=8, columns=10, fallback_manual=True, image_scaling=1.9)



In [11]:
# Just for debugging
sC.save_to_yaml(f"{run_name} no stereo.yaml")

# Stereo Calibration

In [ ]:
# Just for debugging

sC.load_from_yaml(f"{run_name} no stereo.yaml")

In [ ]:
frames = []

for file in stereo_calib[0:]:
    print(os.path.join(calibration_dir, file))
    vL.load_video(os.path.join(calibration_dir, file))
    frames.append(vL[100:110])
opip1ip2 = None

# for testing different stereocalibration_flags without labeling the images new
#sC.load_from_yaml(f"{run_name} no stereo.yaml")
#with open("temp.pk", "rb") as f:
#    opip1ip2 = pk.load(f)

objectpoints, imgpoints_1, imgpoints_2, images = sC.stereo_calibrate(
    frames[:], 
    rows=8, 
    columns=10, 
    undistort=False, 
    opip1ip2=opip1ip2, 
    stereocalibration_flags=cv2.CALIB_USE_INTRINSIC_GUESS,
    image_scaling=3, 
    fallback_manual=True)
# cv2.CALIB_USE_INTRINSIC_GUESS, cv2.CALIB_FIX_INTRINSIC
# cv2.CALIB_FIX_K1 | cv2.CALIB_FIX_K2 | cv2.CALIB_FIX_K3 | cv2.CALIB_FIX_K4 | cv2.CALIB_FIX_K5 | cv2.CALIB_FIX_K6

# Save Calibration to YAML

In [ ]:
sC.save_to_yaml(f"{run_name}.yaml")

sC.save_to_yaml(f"{run_name}_exp.yaml")

In [ ]:
sC.load_from_yaml(f"{run_name}.yaml")